0. Set up notebook to start from repo root

In [1]:
import os
%cd C:\Users\benjo\Documents\Projects\wildlife-camtrap-classification

C:\Users\benjo\Documents\Projects\wildlife-camtrap-classification


1. Data loading and making of manifest
Extracts relevant data from the annotations json and converts and saves into a csv for future usage

In [2]:
images = r"data/cct/images"
annotations = r"data/cct/annotations/caltech_images_20210113.json"
manifest_out = r"data/cct/manifest.csv"
counts_out = r"data/cct/class_counts.csv"
missing_out = r"data/cct/missing_files.txt"

print("cwd:", os.getcwd())

cwd: C:\Users\benjo\Documents\Projects\wildlife-camtrap-classification


In [3]:
from src.data.make_manifest import load_coco, make_manifest_rows, write_manifest, write_counts

coco = load_coco(annotations)
rows,missing = make_manifest_rows(images,coco)
write_manifest(rows,manifest_out)
write_counts(rows,counts_out)
if missing:
    with open(missing_out, "w", encoding="utf-8") as f:
        f.write("\n".join(missing))
print(f'Manifest: {manifest_out}, Rows:{len(rows)}')
print(f'Class Counts : {counts_out}')
if missing:
    print(f'Missing Files: {missing_out}')


Manifest: data/cct/manifest.csv, Rows:243100
Class Counts : data/cct/class_counts.csv


2. Data Preperation

In [4]:
from src.data.split_dataset import load_manifest,split_df,summarise_split

df = load_manifest(manifest_out)
df.head()
missing_per =(df.isna().sum() / len(df)) * 100
print(missing_per)

empty = df['label'].astype(str).str.strip() == ''
print(empty.sum())

rel_path    0.0
label       0.0
location    0.0
dtype: float64
0


From above cells can see in the dataset no missing label,image or path data which would impede training of model, so no intial changes necessary 

In [5]:
import pandas as pd

train,test,val = split_df(df,0.8)

splits_summary = pd.DataFrame([
    summarise_split(train,'Train'),
    summarise_split(test,'Test'),
    summarise_split(val,'Val')
])

splits_summary

c:\Users\benjo\Documents\Projects\wildlife-camtrap-classification\venv\Lib\site-packages\sklearn\model_selection\_split.py:1035: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


,split,rows,cols,n_classes,n_groups,empty_count,empty_pct
0,Train,193815,3,22,110,107123,55.27
1,Test,25428,3,16,16,9667,38.02
2,Val,23857,3,16,14,8955,37.54
